In [19]:
import pandas as pd

df = pd.read_csv('../final_data/clean_model_data.csv')

target = 'nhmliv_self_next_wave' 

basic_controls = ['rafemale', 'rablack','rahispan','raedyrs','agey_e_self',
                  'married_self','agey_e_self_squared', 'raedyrs_squared', 'agey_e_self_squared_X_mstat','agey_e_self_squared_X_female', 'agey_e_self_X_raedyrs',
                  'racohbyr_1','racohbyr_2', 'racohbyr_3', 'racohbyr_4', 'racohbyr_5', 'racohbyr_6','racohbyr_7']

shrs = ['arthre_self', 'cancre_self', 'diabe_self', 'hearte_self', 'hibpe_self', 'lunge_self', 'psyche_self', 'stroke_self','bmi_self', 'smokev_self',]

frailty = ['batha_self', 'dressa_self', 'eata_self',
           'beda_self', 'toilta_self', 'walkra_self', 'walk1a_self', 'walksa_self',
           'shopa_self', 'phonea_self', 'moneya_self', 'mealsa_self', 'medsa_self',
           'mapa_self', 'clim1a_self', 'climsa_self', 'chaira_self', 'stoopa_self',
           'lifta_self', 'armsa_self', 'dimea_self', 'pusha_self', 'sita_self',]

household_characteristics = ['child_household','rameduc', 'rafeduc', 'ravetrn','urban_self', 'rural_self', 'livsib_self', 'momage_self','dadage_self',]

income_assets = ['atotn_household', 'itot_household', 'inpov_household']

In [143]:
female_flag = 0
black_flag = 1
hispan_flag = 0

features_list = basic_controls + shrs + frailty + income_assets + household_characteristics

subset_df = (df[(df['rablack']==black_flag) & (df['rahispan']==hispan_flag) & (df['rafemale']==female_flag)]
                [[target] + features_list]
                .drop(columns=['rablack','rahispan','rafemale']))

# target and features
y = subset_df['nhmliv_self_next_wave'].to_numpy()
X = subset_df.iloc[:, list(df.columns).index('nhmliv_self_next_wave')+1:].to_numpy()

In [144]:
### MODEL 1) VANILLA LOGISTIC REGRESSION

import numpy as np
# Cross Validation
from sklearn.model_selection import StratifiedKFold
# Imputing NaNs
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Oversampling Minority Class
from imblearn.over_sampling import RandomOverSampler
# Model
from sklearn.linear_model import LogisticRegression
# Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss, f1_score
# Preprocessing continuous scales for Logistic Regression
from sklearn.preprocessing import StandardScaler

# Opting to impute missing values with the mean for Logistic Regression
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(max_iter=1000))
])

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=720)

# Results storage
fold_accuracies = []
fold_conf_matrices = []
fold_reports = []
fold_pseudo_r2 = []
fold_f1_scores = []

# Perform Stratified K-Fold CV
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split the dataset into training and testing subsets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Apply oversampling to the training data only
    ros = RandomOverSampler(random_state=720)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    # Fit and Predict
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=False)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Compute log-likelihoods for McFadden's pseudo R^2
    log_likelihood_full = -log_loss(y_train, pipeline.predict_proba(X_train), normalize=False)
    log_likelihood_null = -log_loss(y_train, np.full_like(y_train, y_train.mean()), normalize=False)
    mcfadden_r2 = 1 - (log_likelihood_full / log_likelihood_null)
    fold_pseudo_r2.append(mcfadden_r2)

    # Store results
    fold_accuracies.append(accuracy)
    fold_conf_matrices.append(conf_matrix)
    fold_reports.append(class_report)
    fold_f1_scores.append(f1)

    # Print results for this fold
    # print(f"Fold Accuracy: {accuracy:.4f}")
    # print(f"Fold F1 Score (Weighted): {f1:.4f}")
    # print("Confusion Matrix:")
    # print(conf_matrix)
    # print("Classification Report:")
    # print(class_report)
    # print(f"McFadden's Pseudo R^2: {mcfadden_r2:.4f}")

# Summarize cross-validation results
print("\n=== Cross-Validation Results ===")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation of Accuracy: {np.std(fold_accuracies):.4f}")
print(f"Mean F1 Score (Weighted): {np.mean(fold_f1_scores):.4f}")
print(f"Standard Deviation of F1 Score: {np.std(fold_f1_scores):.4f}")
print(f"Mean McFadden's Pseudo R^2: {np.mean(fold_pseudo_r2):.4f}")
print(f"Standard Deviation of McFadden's Pseudo R^2: {np.std(fold_pseudo_r2):.4f}")


=== Fold 1 ===

=== Fold 2 ===

=== Fold 3 ===

=== Fold 4 ===

=== Cross-Validation Results ===
Mean Accuracy: 0.9877
Standard Deviation of Accuracy: 0.0005
Mean F1 Score (Weighted): 0.9825
Standard Deviation of F1 Score: 0.0002
Mean McFadden's Pseudo R^2: 0.2757
Standard Deviation of McFadden's Pseudo R^2: 0.0187


c:\Users\jacob\anaconda3\envs\health_index\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jacob\anaconda3\envs\health_index\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jacob\anaconda3\envs\health_index\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",